In [ ]:
# pip install opencv-python face_recognition numpy
# installing package for reading the data

In [ ]:
import cv2
import face_recognition
import numpy as np
import pickle
import os

In [ ]:
DATA_FILE = "users_data.pkl"

In [ ]:
# load the data if user is already saved 
def load_users():
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "rb") as f:
            return pickle.load(f)
    return []

In [ ]:
#save the user info 
def save_users(users):
    with open(DATA_FILE, "wb") as f:
        pickle.dump(users, f)

In [ ]:
import time

def register_user(name, age, email, capture_time=10):
    video = cv2.VideoCapture(0)
    start_time = time.time()

    print(f"Camera started. Capturing face in {capture_time} seconds...")

    captured = False

    while True:
        ret, frame = video.read()
        if not ret:
            print("Failed to access camera")
            break

        cv2.imshow("Register Face", frame)

        elapsed_time = time.time() - start_time

        # Auto capture after X seconds
        if elapsed_time >= capture_time:
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            faces = face_recognition.face_locations(rgb)

            if len(faces) != 1:
                print("❌ Please ensure exactly one face is visible")
                break

            encoding = face_recognition.face_encodings(rgb, faces)[0]

            users = load_users()
            users.append({
                "name": name,
                "age": age,
                "email": email,
                "encoding": encoding
            })

            save_users(users)
            print("✅ User registered successfully")
            captured = True
            break

        # Allow emergency exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Registration cancelled")
            break

    video.release()
    cv2.destroyAllWindows()

    if not captured:
        print("⚠️ Registration failed")


In [ ]:
# use the registration feature 
register_user("Sandesh", 22, "sandesh@gmail.com")


In [ ]:
#identify the user
def identify_user():
    users = load_users()
    if len(users) == 0:
        print("No users registered")
        return

    known_encodings = [u["encoding"] for u in users]

    video = cv2.VideoCapture(0)
    print("Scanning face... Press 'q' to quit")

    while True:
        ret, frame = video.read()
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        face_locations = face_recognition.face_locations(rgb)
        face_encodings = face_recognition.face_encodings(rgb, face_locations)

        for encoding in face_encodings:
            matches = face_recognition.compare_faces(known_encodings, encoding)
            face_distances = face_recognition.face_distance(known_encodings, encoding)

            if True in matches:
                best_match = np.argmin(face_distances)
                user = users[best_match]

                print("\n✅ User Identified")
                print("Name:", user["name"])
                print("Age:", user["age"])
                print("Email:", user["email"])

                video.release()
                cv2.destroyAllWindows()
                return

        cv2.imshow("Identify Face", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()

    


In [ ]:
identify_user()
